In [ ]:
import numpy as np
import importlib, os, datetime
from sus.protocol_designer import Potential, System
from sus.protocol_designer.protocol import Injected_Protocol
# from sus.protocol_designer import Potential
from sus.protocol_designer.protocol import sequential_protocol
from IPython import display
from IPython.display import HTML, Image
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation, PillowWriter


from quick_sim import setup_sim

# import edward_tools.fq_runner as fq_runner
from edward_tools.simple_1D_potential import flux_qubit_1D_non_linear_approx_pot, flux_qubit_1D_non_linear_approx_force
from edward_tools.visualization import animate_sim_flux_qubit, plotFidelityBarChart, separate_by_state_2
from edward_tools.initial_state_sampling import extra_constraint_00_and_11_only
from NAND_PARAMETERS import *
import importlib, pathlib

import kyle_tools as kt
import matplotlib.pyplot as plt
from sqlite_database import DatabaseManager

from edward_tools import coupled_fq_protocol_library, cfq_runner
from edward_tools import coupled_fq_protocol_library

import edward_tools.cfq_batch_sweep as cfq_batch_sweep
import edward_tools.Analysis_tool.general_analysis_tools as general_analysis_tool
# from edward_tools.Analysis_tool.general_analysis_tools import show_phi_dc_with_time
import edward_tools.Analysis_tool.minimum_value_of_potential as minimum_value_of_potential
from edward_tools.couple_flux_qubit_metrics import fidelityEvaluation
from edward_tools import visualization

coupled_fq_protocol_library = importlib.reload(coupled_fq_protocol_library)
create_system = coupled_fq_protocol_library.create_system
get_potential_shot_at_different_t = coupled_fq_protocol_library.get_potential_shot_at_different_t
get_potential_shot_at_different_t_1D = coupled_fq_protocol_library.get_potential_shot_at_different_t_1D
create_simple_protocol_parameter_dict = coupled_fq_protocol_library.create_simple_protocol_parameter_dict
coupled_fq_runner = importlib.reload(cfq_runner)
coupled_fq_protocol_library = importlib.reload(coupled_fq_protocol_library)
create_system = coupled_fq_protocol_library.create_system
get_potential_along_a_1D_cutline = coupled_fq_protocol_library.get_potential_along_a_1D_cutline
plotCutlines = coupled_fq_protocol_library.plotCutlines

# show import data

In [ ]:
import pathlib

# A path string with a tilde
tilde_path = "~/protocolopt/sim_data/simulations.db"
tilde_path = "~/Project/protocolopt/sim_data/simulations.db"


expanded_path = pathlib.Path(tilde_path).expanduser()
sqliteDatabase = DatabaseManager(expanded_path)

# experiment select
 

experiment_id = "656e98ba9399facd1cf40c9c6a70c09887a5a6b097bf773e83815036e4c4c2c2" # Original Protocol
experiment_id = "30c78d77fdba17f06a6e2ca9ce449966b9dbff40eaaa801eeaa1baae8820864e" # optimizer

experiemnt_data_array = sqliteDatabase.get_all_simulations()
experiemnt_data = [data for data in experiemnt_data_array if data['experiment_id'] == experiment_id][0]

In [ ]:
has_velocity = True

PHI_0 = 2.067833848 * 1e-15
k_B = 1.38e-23
T = 4.2
# T = 7
k_BT = k_B * T

C_factor = 1
L_factor = 1
R_factor = experiemnt_data["R"]
# I_m_factor = 50
# I_m_factor = 15
I_m_factor = 0
time_scale = 1.0


I_p_1, I_p_2 = 5e-6 , 5e-6  # Amp
I_m_1, I_m_2 = 7e-9 * I_m_factor, 7e-9 * I_m_factor                           # Amp
R_1, R_2 = 100 * R_factor, 100 * R_factor                                         # ohm

# L_1, L_2 = 140e-12 * L_factor, 140e-12 * L_factor                             # H 
L_1, L_2 = 5e-12 * L_factor, 5e-12 * L_factor                             # H 
C_1, C_2 = 1e-12 * C_factor, 1e-12 * C_factor                             # F



freq = 1/np.sqrt(C_1 * L_1)

characteristic_time = np.sqrt(C_1 * C_factor * L_1 * L_factor)
print(f"freq = {freq / 1e9:.3f}GHz, characteristic_time  = {characteristic_time * 1e12:.3f}ps")


In [ ]:

m_c = C_1
m_1 = C_1
m_2 = C_2
x_c = PHI_0 / (2 * np.pi)
time_scale_factor = 1
t_c = np.sqrt(L_1 * C_1)
v_c = x_c / t_c


U0_1 = m_c * x_c**2 / t_c**2 / k_BT
U0_2 = m_2 * x_c**2 / t_c**2 / k_BT
kappa_1, kappa_2, kappa_3, kappa_4 = 1/U0_1, 1/U0_1, 1/U0_1, 1/U0_1

lambda_1 = 2 * np.sqrt(L_1 * C_1) / (C_1 * R_1)
theta_1  = 1
eta_1    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(2 * kappa_1 / 1**2)

lambda_2 = 2 * np.sqrt(L_1 * C_1) / (C_2 * R_2)
theta_2  = 1 / (C_2/C_1)
eta_2    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(2 * kappa_2 * (R_1 * C_1**2) / (R_2 * C_2**2))

lambda_3 = 2 * np.sqrt(L_1 * C_1) / (C_1 * R_1)
theta_3  = 4
eta_3    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(8 * kappa_3)

lambda_4 = 2 * np.sqrt(L_1 * C_1) / (C_2 * R_2)
theta_4  = 4 / (C_2/C_1)
eta_4    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(8 * kappa_4 * (R_1 * C_1**2) / (R_2 * C_2**2))

gamma = 16


beta_1 = 2 * np.pi * L_1 * I_p_1 / PHI_0; 
beta_2 = 2 * np.pi * L_2 * I_p_2 / PHI_0;

beta_1 = 2.3
beta_2 = 2.3

d_beta_1 = 2 * np.pi * L_1 * I_m_1 / PHI_0; 
d_beta_2 = 2 * np.pi * L_2 * I_m_2 / PHI_0;

_lambda = np.array([lambda_2])
_theta  = np.array([theta_2])
_eta  =   np.array([eta_2])

### parameter setting

In [ ]:
"""
# step 0: modify parameters
- All the parameters are stored in a separate file PARAMETER_INPUT
- You can override some of the parameters here.
"""
params = {}
params['N'] = 10000
params['dt'] = experiemnt_data["dt_LC"]
params['lambda'] = 1
params['beta'] = 1
params['sim_params'] = [_lambda, _theta, _eta]
params['target_work'] = None
params['applyOffset'] = False
params['measureWorkWithOffset'] = False
params['monitor_work_dist_in_whole_process'] = True # To monitor the work process
params['comment'] = "Experiment 8 (2024/3/17): 4 well, with no compensation for asym, 1/5000"
params['capacitance'] = [C_1]
params['mass'] = np.array([1])
params['v_c'] = x_c/t_c
params['k_BT'] = k_BT
params['U0'] = U0_1
params['as_step'] = np.s_[::500] # the time step to skep for the all_state
params['percentage'] = 1 # For what percentage of the total sample do you want to keep in the output all_state


In [ ]:
params['dt']

In [ ]:
"""
# step 2: Define initial condition and protocol
"""
manual_domain=[np.array([-5, -5]), np.array([5, 5])]
    
initial_parameter_dict = {
        "U0_1": U0_1,     "gamma": gamma,  "beta_1": beta_1,   "d_beta_1": d_beta_1,
        "phi_1x": 0,  "phi_1xdc": 0, 'x_c': x_c, 'shift': 0
}


In [ ]:
"""
# step 1: Define potential
"""
coupled_fq_default_param = [U0_1, gamma, beta_1, d_beta_1, 0, 0, x_c]
[phi_1_bound, phi_1dc_bound] = np.array([4, 4])/time_scale_factor
contour_range = [300, 2000]

coupled_fq_domain = [[-phi_1_bound], [phi_1_bound]]

coupled_fq_pot = Potential(flux_qubit_1D_non_linear_approx_pot, flux_qubit_1D_non_linear_approx_force, 8, 1, default_params = initial_parameter_dict,  relevant_domain = coupled_fq_domain)

In [ ]:
zeroDissipation = False
zeroDissipation = True
saveAllStates = True

params['sim_params'] = [_lambda, _theta, _eta]

if zeroDissipation:
    params['sim_params'] = [_lambda * 0, _theta, _eta * 0]

# bookmark
initial_parameter_dict["phi_1xdc"] = 0
initial_parameter_dict["phi_1x"]   = 0

In [ ]:
import torch

uid = experiemnt_data["experiment_id"]

data_location_tilde_path = "~/Project/protocolopt/sim_data"
data_location_expanded_path = pathlib.Path(data_location_tilde_path).expanduser()

microstate_paths_load = torch.load(f"{data_location_expanded_path}/{uid}_microstate_paths.pt").detach().cpu().numpy()
potential_val_load = torch.load(f"{data_location_expanded_path}/{uid}_potential_val.pt").detach().cpu().numpy()
protocol_tensor_load = torch.load(f"{data_location_expanded_path}/{uid}_protocol_tensor.pt").detach().cpu().numpy()

# protocol_tensor_load[0, :] = 0
# protocol_tensor_load[1, 1:-1] = np.pi
# protocol_tensor_load[1, :] = 0

t_duration =  experiemnt_data["t_LC_duration"]

# params['dt'] = 1/100
time_steps = int(t_duration / params['dt'])
injected_protocol = Injected_Protocol(t_duration,  params['dt'], protocol_tensor_load, coupled_fq_default_param, replace_index = [4, 5])

# create cfqr object

In [ ]:
init_state_used = np.load("saved_init_state/initial_state_1D.npy")
# init_state_used = np.full_like(init_state_used, 1)
# init_state_used[:, 0, 1] = 0

In [ ]:
from edward_tools.fq_runner import fluxQubitRunner
fq_runner = fluxQubitRunner(potential=coupled_fq_pot, params=params, potential_default_param_dict=initial_parameter_dict, storage_protocol=injected_protocol, computation_protocol=injected_protocol)

fq_runner.initialize_sim()
fq_runner.set_sim_attributes(init_state=init_state_used)

# plt.hist(fq_runner.init_state, bins = 100)

In [ ]:
simResult = fq_runner.run_sim(init_state=init_state_used, manual_domain = [-4, 4], axes = [0, 1], percentage = 1)

In [ ]:
all_states = fq_runner.sim.output.all_state["states"]
mean_KE_i = np.mean(fq_runner.system.get_kinetic_energy(all_states[:, 0, ...]))
mean_KE_f = np.mean(fq_runner.system.get_kinetic_energy(all_states[:, -1, ...]))
print(f"initial_KE = {mean_KE_i}, final_KE = {mean_KE_f}")


In [ ]:


plt.hist(all_states[:, 0, 0, 0], bins = 100)
bins = np.linspace(-4, 4, 200)
phi_1_array = np.linspace(-4, 4, 100)

fig, ax = plt.subplots()
pdf_ax = ax.twinx()
p_0_index = all_states[:, 0, 0, 0] < 0
p_1_index = all_states[:, 0, 0, 0] > 0
p_0_color, p_1_color = "red", "blue"

# x_array = torch.linspace(-4, 4, 50, device=device).reshape(-1, 1)
# potential_at_t_0 = potential.get_potential_value(x_array, protocol_tensor_load, 0)
params_at_0 = fq_runner.protocol.get_params(0)
potential_at_t_0 = flux_qubit_1D_non_linear_approx_pot(phi_1_array, params_at_0)


ax.plot(phi_1_array, potential_at_t_0 -  min(potential_at_t_0), color='red')
plt.hist(fq_runner.sim.output.all_state["states"][p_0_index, 0, 0, 0], bins = bins, color = p_0_color)
plt.hist(fq_runner.sim.output.all_state["states"][p_1_index, 0, 0, 0], bins = bins, color = p_1_color)
pdf_ax.get_yaxis().set_visible(False)
pdf_ax.set_xlim(-4, 4)

def update(i):
    # for each frame, update the data stored on each artist.
    ax.clear()
    pdf_ax.clear()
    ax.set_ylabel('Potential (k_BT)')

    params_at_t_i = fq_runner.protocol.get_params(i * params["dt"])
    potential_at_t_i = flux_qubit_1D_non_linear_approx_pot(phi_1_array, params_at_t_i)
    potential_at_t_i = potential_at_t_i - min(potential_at_t_i)
    ax.plot(phi_1_array, potential_at_t_i, color='red')
    pdf_ax.hist(all_states[p_0_index, i, 0, 0], bins = bins, color = p_0_color)
    pdf_ax.hist(all_states[p_1_index, i, 0, 0], bins = bins, color = p_1_color)
    pdf_ax.set_xlim(-4, 4)
    ax.set_title(f"old simulator (t = {params['dt'] * i:.3g})")

# animation_frames = microstate_paths_load.shape[2]-1
animation_frames = range(0, all_states.shape[1], 5)
ani = animation.FuncAnimation(fig, update, frames=animation_frames, interval = 50)
# ani = animation.FuncAnimation(fig=fig, func=update, frames=time_steps, interval=30)
plt.close()
# # HTML(ani.to_jshtml())
HTML(ani.to_html5_video())

In [ ]:
plt.hist(fq_runner.sim.work_dist_array, bins = 100)
plt.title(f"mean work = {np.mean(fq_runner.sim.work_dist_array):.3g} k_BT") 
plt.xlabel("W (k_BT)")
plt.ylabel("count")

In [ ]:
fq_runner.sim.work_dist_array.shape

In [ ]:
time_array = np.arange(0, int(t_duration /params['dt'])) * params['dt']

In [ ]:
chosen_trajectory = 10

KE_array = fq_runner.system.get_kinetic_energy(all_states[chosen_trajectory, 0:-1, ...])
PE_array = [fq_runner.system.get_potential(all_states[chosen_trajectory, i, ...], t) for i, t in enumerate(time_array)]
TE_array = KE_array + PE_array


# PE_array = 
# total_energy_array = KE_array[:, 1:] + PE_array
# plt.plot(time_array, PE_array)
plt.plot(time_array, KE_array)
# plt.plot(time_array, TE_array)


# PE_array = potential_val_load.detach().cpu().numpy() * U_0
# total_energy_array = KE_array[:, 1:] + PE_array

In [ ]:
fq_runner.sim.output.all_W['values'].shape

In [ ]:
np.mean(KE_array)

In [ ]:
tilde_path = "~/protocolopt/sim_data/simulations.db"
expanded_path = pathlib.Path(tilde_path).expanduser()
sqliteDatabase = DatabaseManager(expanded_path)

# experiment select
experiment_id = "656e98ba9399facd1cf40c9c6a70c09887a5a6b097bf773e83815036e4c4c2c2" # CRAZY protocol
experiemnt_data_array = sqliteDatabase.get_all_simulations()
experiemnt_data = [data for data in experiemnt_data_array if data['experiment_id'] == experiment_id][0]

In [ ]:
has_velocity = True

PHI_0 = 2.067833848 * 1e-15
k_B = 1.38e-23
T = 4.2
# T = 7
k_BT = k_B * T

C_factor = 1
L_factor = 1
R_factor = experiemnt_data['R']
# I_m_factor = 50
# I_m_factor = 15
I_m_factor = 0
time_scale = 1.0


I_p_1, I_p_2 = 5e-6 , 5e-6  # Amp
I_m_1, I_m_2 = 7e-9 * I_m_factor, 7e-9 * I_m_factor                           # Amp
R_1, R_2 = 1 * R_factor, 1 * R_factor                                         # ohm
C_1, C_2 = 1e-12 * C_factor, 1e-12 * C_factor                             # F

L_1, L_2 = 140e-12 * L_factor, 140e-12 * L_factor                             # H 
L_1, L_2 = 5e-12 * L_factor, 5e-12 * L_factor                             # H 
freq = 1/np.sqrt(C_1 * L_1)
characteristic_time = np.sqrt(C_1 * C_factor * L_1 * L_factor)
print(f"freq = {freq / 1e9:.3f}GHz, characteristic_time  = {characteristic_time * 1e12:.3f}ps")


In [ ]:

m_c = C_1
m_1 = C_1
m_2 = C_2
x_c = PHI_0 / (2 * np.pi)
time_scale_factor = 1
t_c = np.sqrt(L_1 * C_1)
v_c = x_c / t_c


U0_1 = m_c * x_c**2 / t_c**2 / k_BT
U0_2 = m_2 * x_c**2 / t_c**2 / k_BT
kappa_1, kappa_2, kappa_3, kappa_4 = 1/U0_1, 1/U0_1, 1/U0_1, 1/U0_1

lambda_1 = 2 * np.sqrt(L_1 * C_1) / (C_1 * R_1)
theta_1  = 1
eta_1    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(2 * kappa_1 / 1**2)

lambda_2 = 2 * np.sqrt(L_1 * C_1) / (C_2 * R_2)
theta_2  = 1 / (C_2/C_1)
eta_2    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(2 * kappa_2 * (R_1 * C_1**2) / (R_2 * C_2**2))

lambda_3 = 2 * np.sqrt(L_1 * C_1) / (C_1 * R_1)
theta_3  = 4
eta_3    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(8 * kappa_3)

lambda_4 = 2 * np.sqrt(L_1 * C_1) / (C_2 * R_2)
theta_4  = 4 / (C_2/C_1)
eta_4    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(8 * kappa_4 * (R_1 * C_1**2) / (R_2 * C_2**2))

gamma = 16


beta_1 = 2 * np.pi * L_1 * I_p_1 / PHI_0; 
beta_2 = 2 * np.pi * L_2 * I_p_2 / PHI_0;

beta_1 = 2.3
beta_2 = 2.3

d_beta_1 = 2 * np.pi * L_1 * I_m_1 / PHI_0; 
d_beta_2 = 2 * np.pi * L_2 * I_m_2 / PHI_0;

_lambda = np.array([lambda_2])
_theta  = np.array([theta_2])
_eta  =   np.array([eta_2])

### parameter setting

In [ ]:
"""
# step 0: modify parameters
- All the parameters are stored in a separate file PARAMETER_INPUT
- You can override some of the parameters here.
"""
params = {}
params['N'] = 4000
params['dt'] = experiemnt_data['dt_LC']
params['lambda'] = 1
params['beta'] = 1
params['sim_params'] = [_lambda, _theta, _eta]
params['target_work'] = None
params['applyOffset'] = False
params['measureWorkWithOffset'] = False
params['monitor_work_dist_in_whole_process'] = True # To monitor the work process
params['comment'] = "Experiment 8 (2024/3/17): 4 well, with no compensation for asym, 1/5000"
params['capacitance'] = [C_1]
params['mass'] = np.array([1])
params['v_c'] = x_c/t_c
params['k_BT'] = k_BT
params['U0'] = U0_1
params['as_step'] = np.s_[::500] # the time step to skep for the all_state
params['percentage'] = 1 # For what percentage of the total sample do you want to keep in the output all_state


In [ ]:
"""
# step 2: Define initial condition and protocol
"""
manual_domain=[np.array([-5, -5]), np.array([5, 5])]
    
initial_parameter_dict = {
        "U0_1": U0_1,     "gamma": gamma,  "beta": beta_1,   "d_beta": d_beta_1,
        "phi_1x": 0,  "phi_1xdc": 0, 'x_c': x_c
}

In [ ]:
"""
# step 1: Define potential
"""
coupled_fq_default_param = [U0_1, gamma, beta_1, d_beta_1, 0, 0, x_c]
[phi_1_bound, phi_1dc_bound] = np.array([4, 4])/time_scale_factor
contour_range = [300, 2000]

coupled_fq_domain = [[-phi_1_bound], [phi_1_bound]]

# coupled_fq_pot = Potential(coupled_flux_qubit_pot_with_offset_at_00_xy, coupled_flux_qubit_force, 14, 4,\
#                            default_params = coupled_fq_default_param,  relevant_domain = coupled_fq_domain)

coupled_fq_pot = Potential(flux_qubit_1D_non_linear_approx_pot, flux_qubit_1D_non_linear_approx_force, 7, 1, default_params = initial_parameter_dict,  relevant_domain = coupled_fq_domain)

# coupled_fq_pot = Potential(coupled_flux_qubit_non_linear_approx_pot_special, coupled_flux_qubit_non_linear_approx_force_special, 14, 4,\
                           # default_params = initial_parameter_dict,  relevant_domain = coupled_fq_domain)

In [ ]:
zeroDissipation = False
# zeroDissipation = True

In [ ]:
saveAllStates = True

params['sim_params'] = [_lambda, _theta, _eta]

if zeroDissipation:
    params['sim_params'] = [_lambda * 0, _theta, _eta * 0]



# bookmark
initial_parameter_dict["phi_1xdc"] = 0
initial_parameter_dict["phi_1x"]   = 0

In [ ]:

# longer CE
t_duration = experiemnt_data['t_LC_duration']

ratio_array = [0.998] * 5
# ratio_array = [0.9, 0.9, 0.9, 0.9, 0.9]
protocol_list_variable_duration = [
    # forward
    
    create_CE_Protocol(10, {
    "phi_1x": 0, "phi_2x": 0, "mu_12": 0, \
    "phi_1xdc": np.pi, "phi_2xdc": 0, "name":"decrease k"
    }),
]

protocol_list = protocol_list_variable_duration

In [ ]:
protocol_list_variable_duration

In [ ]:
# protocol_list_variable_duration = [
#     # forward
#     create_CE_Protocol(10, {
#     "phi_1_x": 0, "phi_2_x": 0, "M_12": 0, \
#     "phi_1_dcx": 0, "phi_2_dcx": 0, "name":"decrease k"
#     }),
#     create_CE_Protocol(20, {
#     "phi_1_x": 0, "phi_2_x": 0, "M_12": 0, \
#     "phi_1_dcx": 0, "phi_2_dcx": 0.5, "name":"decrease k"
#     }),
#     create_CE_Protocol(30, {
#     "phi_1_x": 0, "phi_2_x": 0, "M_12": 0, \
#     "phi_1_dcx": 0, "phi_2_dcx": 1.0, "name":"decrease k"
#     }),
#    create_CE_Protocol(40, {
#     "phi_1_x": 0, "phi_2_x": 0, "M_12": 0, \
#     "phi_1_dcx": 0, "phi_2_dcx": 1.5, "name":"decrease k"
#     }),
#    create_CE_Protocol(50, {
#     "phi_1_x": 0, "phi_2_x": 0, "M_12": 0, \
#     "phi_1_dcx": 0, "phi_2_dcx": 2.0, "name":"decrease k"
#     })
#     ]
# protocol_list = protocol_list_variable_duration

# create cfqr object

In [ ]:
protocol_key = ['U0_1', 'gamma', 'beta', 'd_beta', 'phi_1x', 'phi_1xdc', 'x_c']


def customizedProtocol(initial_values_dict, protocol_list, normalized = False):
    protocol_parameter_dict = {key: [value] for key, value in initial_values_dict.items()}
    protocol_parameter_dict["t"] = [0.0]


    for item in protocol_list:
        # add the duration to the time entry of the protocol_parameter_dict
        protocol_parameter_dict["t"].append(protocol_parameter_dict["t"][-1] + item["duration"])

        for key in protocol_key:
            if key in item.keys(): # to check which key is present in the protocol_list_item.
                protocol_parameter_dict[key].append(item[key])
            else:
                protocol_parameter_dict[key].append(protocol_parameter_dict[key][-1])

    if normalized:
        protocol_parameter_dict["t"] = np.array(protocol_parameter_dict["t"])/ np.max(protocol_parameter_dict["t"])

    return protocol_parameter_dict


def create_system(protocol_parameter_dict, domain = None, modifiedFunction = None):
    """
    This function is used to produce the storage and computation protocol

    input:
    1. comp_protocol_parameter_dict:
    - a dictionary contains the an array of time, which represents the time point at which the protocol is changed
    - the key is the name of the parameter
    - for parameters, they are arrays containing the value of the parameter at the particular time point

    2. modifiedFunction:
    - To modify the simple linear parametrization of the value
    - You can use more complex method to get the values such as calculate the value of phi_1x based on the value of other parameters
    
    output:
    1. comp_prototocl: the protocol for the computation system
    2. storage_protocol: : the protocol for the equilibrium system
    """
    # storage protocol, just take the first and last element of each term in the protocol key from the dict to form the storage protocol
    storage_t = (protocol_parameter_dict["t"][0], protocol_parameter_dict["t"][-1])
    storage_protocol_parameter_time_series = [np.array([protocol_parameter_dict[key][0], protocol_parameter_dict[key][-1]]) for key in protocol_key]
    storage_protocol_parameter_time_series = np.array(storage_protocol_parameter_time_series)
    storage_protocol = Protocol(storage_t, storage_protocol_parameter_time_series)
    storage_protocol.modifiedFunction = None
    # computation protocol, this part form the time series of the comp_protocol and join them to form Compound_Protocol
    comp_protocol_array = []
    comp_t = protocol_parameter_dict["t"]
    comp_protocol_parameter_time_series = [protocol_parameter_dict[key] for key in protocol_key]
    comp_protocol_parameter_time_series = np.array(comp_protocol_parameter_time_series).T

    
    if modifiedFunction == None:
        modifiedFunction = [None for _ in range(len(comp_t)-1)]
    
    
    for i in range(len(comp_t)-1):
        n_th_comp_time_array = (comp_t[i], comp_t[i+1])
        n_th_comp_protocol_parameter_array = np.array([comp_protocol_parameter_time_series[i], comp_protocol_parameter_time_series[i+1]]).T # in the form of array of [(p_n_i, p_n_f)]
        _p = Protocol(n_th_comp_time_array, n_th_comp_protocol_parameter_array)
        
        _p.modifiedFunction = modifiedFunction[i]
        comp_protocol_array.append(_p)
    comp_protocol = Compound_Protocol(comp_protocol_array)
    comp_protocol.protocol_array  = comp_protocol_array

    return storage_protocol, comp_protocol

In [ ]:
protocol_list

In [ ]:
from edward_tools.fq_runner import fluxQubitRunner

computation_protocol_parameter_dict = customizedProtocol(initial_parameter_dict, \
                                                                    protocol_list)
storage_protocol, comp_protocol = create_system(computation_protocol_parameter_dict, modifiedFunction = None)



fq_runner = fluxQubitRunner(potential=coupled_fq_pot, params=params, potential_default_param_dict=initial_parameter_dict, storage_protocol=storage_protocol, computation_protocol=comp_protocol)


In [ ]:
comp_protocol.protocol_array[0].get_params(10)

In [ ]:
simResult = fq_runner.run_sim()

In [ ]:
fq_runner.protocol.get_params(0)

In [ ]:
phi_1_array = np.linspace(-4, 4, 100)
params_at_0 = fq_runner.protocol.get_params(0)
flux_qubit_1D_non_linear_approx_pot(phi_1_array, params_at_0)

In [ ]:
all_states = fq_runner.sim.output.all_state["states"]

plt.hist(all_states[:, 0, 0, 0], bins = 100)
bins = np.linspace(-4, 4, 200)
phi_1_array = np.linspace(-4, 4, 100)

fig, ax = plt.subplots()
pdf_ax = ax.twinx()

# x_array = torch.linspace(-4, 4, 50, device=device).reshape(-1, 1)
# potential_at_t_0 = potential.get_potential_value(x_array, protocol_tensor_load, 0)
params_at_0 = fq_runner.protocol.get_params(0)
potential_at_t_0 = flux_qubit_1D_non_linear_approx_pot(phi_1_array, params_at_0)
potential_shift = min(potential_at_t_0)
potential_at_t_0 = potential_at_t_0 - potential_shift

ax.plot(phi_1_array, potential_at_t_0, color='red')
plt.hist(fq_runner.sim.output.all_state["states"][:, 0, 0, 0], bins = bins)
pdf_ax.get_yaxis().set_visible(False)
pdf_ax.set_xlim(-4, 4)

def update(i):
    # for each frame, update the data stored on each artist.
    ax.clear()
    pdf_ax.clear()
    ax.set_ylabel('Potential (k_BT)')
    new_data = all_states[:, i, 0, 0]
    params_at_t_i = fq_runner.protocol.get_params(i * params["dt"])
    potential_at_t_i = flux_qubit_1D_non_linear_approx_pot(phi_1_array, params_at_t_i)
    potential_at_t_i = potential_at_t_i - potential_shift
    # potential_at_t_i = potential.get_potential_value(x_array, protocol_tensor_load, i)
    # print(potential_at_t_i)
    ax.plot(phi_1_array, potential_at_t_i, color='red')
    pdf_ax.hist(new_data, bins = bins)
    pdf_ax.set_xlim(-4, 4)

# animation_frames = microstate_paths_load.shape[2]-1
animation_frames = range(0, all_states.shape[1]-1, 50)
ani = animation.FuncAnimation(fig, update, frames=animation_frames, interval = 50)
# ani = animation.FuncAnimation(fig=fig, func=update, frames=time_steps, interval=30)
plt.close()
# # HTML(ani.to_jshtml())
HTML(ani.to_html5_video())

In [ ]:
params["dt"]